In [23]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn

In [24]:
from keras.datasets import cifar10

In [25]:
import tensorflow as tf
print(tf.__version__)



2.2.0


In [26]:
(X_train, y_train) , (X_test, y_test) = cifar10.load_data()

In [27]:
print('X_train shape - ',X_train.shape)
print('X_test shape - ',X_test.shape)
print('y_train shape - ',y_train.shape)
print('y_test shape - ',y_test.shape)

X_train shape -  (50000, 32, 32, 3)
X_test shape -  (10000, 32, 32, 3)
y_train shape -  (50000, 1)
y_test shape -  (10000, 1)


In [28]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train/255
X_test = X_test/255

In [29]:
import keras
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [30]:
Input_shape = X_train.shape[1:]
Input_shape

(32, 32, 3)

In [31]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,AveragePooling2D,MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import BatchNormalization

In [32]:
cnn_model = Sequential()
cnn_model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', kernel_initializer='he_uniform', padding='same',input_shape = Input_shape))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', kernel_initializer='he_uniform',padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(2,2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', kernel_initializer='he_uniform', padding='same',input_shape = Input_shape))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', kernel_initializer='he_uniform',padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(2,2))
cnn_model.add(Dropout(0.3))


cnn_model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', kernel_initializer='he_uniform',padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', kernel_initializer='he_uniform',padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(AveragePooling2D(2,2))
cnn_model.add(Dropout(0.4))

cnn_model.add(Flatten())

cnn_model.add(Dense(units = 1024, activation = 'relu', kernel_initializer='he_uniform'))
cnn_model.add(BatchNormalization())
cnn_model.add(Dropout(0.5))

cnn_model.add(Dense(units = 10, activation = 'softmax'))

In [33]:
cnn_model.compile(loss = 'categorical_crossentropy', optimizer = Adam(lr=0.001,beta_1=0.9,beta_2=0.999), metrics = ['accuracy'])

In [34]:
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        1

In [35]:
history = cnn_model.fit(X_train, y_train, batch_size = 64, epochs = 20, shuffle = True,validation_data=(X_test,y_test))

Epoch 1/20
782/782 [==============================] - 7s 9ms/step - loss: 1.6660 - accuracy: 0.4643 - val_loss: 1.2456 - val_accuracy: 0.5748
Epoch 2/20
782/782 [==============================] - 7s 9ms/step - loss: 1.0695 - accuracy: 0.6279 - val_loss: 0.9072 - val_accuracy: 0.6892
Epoch 3/20
782/782 [==============================] - 7s 9ms/step - loss: 0.8701 - accuracy: 0.6932 - val_loss: 0.7347 - val_accuracy: 0.7476
Epoch 4/20
782/782 [==============================] - 7s 8ms/step - loss: 0.7636 - accuracy: 0.7329 - val_loss: 0.7368 - val_accuracy: 0.7560
Epoch 5/20
782/782 [==============================] - 7s 9ms/step - loss: 0.6867 - accuracy: 0.7604 - val_loss: 0.7304 - val_accuracy: 0.7600
Epoch 6/20
782/782 [==============================] - 7s 9ms/step - loss: 0.6346 - accuracy: 0.7810 - val_loss: 0.6368 - val_accuracy: 0.7837
Epoch 7/20
782/782 [==============================] - 7s 8ms/step - loss: 0.5903 - accuracy: 0.7955 - val_loss: 0.5626 - val_accuracy: 0.8117
Epoch 

In [36]:
evaluation = cnn_model.evaluate(X_test, y_test)
print('Test Accuracy: {}'.format(evaluation[1]))

313/313 [==============================] - 1s 3ms/step - loss: 0.4794 - accuracy: 0.8578
Test Accuracy: 0.8578000068664551


In [37]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
                            
                            
                            width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.1,
                             zoom_range=0.1,
                            horizontal_flip = True,
                            
                             )
datagen.fit(X_train)

In [38]:
cnn_model.fit_generator(datagen.flow(X_train, y_train, batch_size = 64), epochs = 250)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/250
782/782 [==============================] - 23s 30ms/step - loss: 0.6223 - accuracy: 0.7913
Epoch 2/250
782/782 [==============================] - 23s 30ms/step - loss: 0.5503 - accuracy: 0.8117
Epoch 3/250
782/782 [==============================] - 23s 30ms/step - loss: 0.5368 - accuracy: 0.8153
Epoch 4/250
782/782 [==============================] - 23s 30ms/step - loss: 0.5151 - accuracy: 0.8208
Epoch 5/250
782/782 [==============================] - 23s 30ms/step - loss: 0.5073 - accuracy: 0.8253
Epoch 6/250
782/782 [==============================] - 24s 31ms/step - loss: 0.4914 - accuracy: 0.8299
Epoch 7/250
782/782 [==============================] - 24s 31ms/step - loss: 0.4894 - accuracy: 0.8317
Epoch 8/250
782/782 [==============================] - 24s 31ms/step - loss: 0.4767 - accuracy: 0.8359
Epoch 9/250
782/782 [==============================] - 24s 30ms/step - loss: 0.4710 - accuracy: 0.83

In [39]:
score = cnn_model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.3222 - accuracy: 0.9070


In [40]:
print('Test accuracy', score[1])

Test accuracy 0.9070000052452087


In [41]:
score_test= cnn_model.evaluate(X_train, y_train)

1563/1563 [==============================] - 5s 3ms/step - loss: 0.0455 - accuracy: 0.9847


In [42]:
print('Train accuracy', score_test[1])

Train accuracy 0.98471999168396


In [44]:
import os.path

In [46]:
MODEL_DIR = '/content/drive/My Drive/Colab Notebooks/cifar model'
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    cnn_model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = /content/drive/My Drive/Colab Notebooks/cifar model/1

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/drive/My Drive/Colab Notebooks/cifar model/1/assets

Saved model:
ls: cannot access '/content/drive/My': No such file or directory
ls: cannot access 'Drive/Colab': No such file or directory
ls: cannot access 'Notebooks/cifar': No such file or directory
ls: cannot access 'model/1': No such file or directory


In [47]:
!saved_model_cli show --dir {export_path} --all

usage: saved_model_cli [-h] [-v] {show,run,scan,convert,aot_compile_cpu} ...
saved_model_cli: error: unrecognized arguments: Drive/Colab Notebooks/cifar model/1


In [54]:
# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0   151k      0 --:--:-- --:--:-- --:--:--  151k
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Err:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease
  The following signatures were invalid: EXPKEYSIG 544B7F63BF9E4D5F Tensorflow Serving Developer (Tensorflow Serving APT repository key) <tensorflow-serving-dev@googlegroups.com>
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 k

In [56]:
!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package tensorflow-model-server
